<H1>ФБ-31 Гек Роман</H1>
<h2>Лабораторна робота 2</h2>

Блок 1: Імпорт бібліотек та підготовка середовища

In [1]:
import os
import urllib.request
import pandas as pd
import numpy as np
from datetime import datetime
import glob
from prettytable import PrettyTable

print("Бібліотеки успішно імпортовано")

Бібліотеки успішно імпортовано


Блок 2: Ф-я для завантаження даних VHI за індексом області

In [2]:
def download_vhi_data(province_id, output_dir="vhi_data", year1=1981, year2=2024):
    # Перевіряю чи існує директорія (ні - створюю)
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        print(f"Створено директорію {output_dir}")
    
    # URL для даних
    url = f'https://www.star.nesdis.noaa.gov/smcd/emb/vci/VH/get_TS_admin.php?country=UKR&provinceID={province_id}&year1={year1}&year2={year2}&type=Mean'
    
    try:
        # Перевіряю, чи існують вже файли для області
        existing_files = [f for f in os.listdir(output_dir) if f.startswith(f'vhi_id_{province_id}_')]
        
        if existing_files:
            print(f"Файл для області {province_id} вже існує: {existing_files[0]}")
            return os.path.join(output_dir, existing_files[0])
        
        # Завантажую дані з URL
        vhi_url = urllib.request.urlopen(url)
        data = vhi_url.read()
        
        # Додаю дату і час до імені файлу
        now = datetime.now()
        date_time = now.strftime("%d%m%Y%H%M%S")
        filename = f'vhi_id_{province_id}_{date_time}.csv'
        filepath = os.path.join(output_dir, filename)
        
        # Записую дані у файл
        with open(filepath, 'wb') as out:
            out.write(data)
        
        print(f"Дані для області {province_id} успішно завантажено у {filename}")
        return filepath
        
    except Exception as e:
        print(f"Помилка при завантаженні даних для області {province_id}: {e}")
        return None

Блок 3: Ф-я для завантаження даних VHI для всіх областей України

In [3]:
def download_all_provinces(output_dir="vhi_data"):

    filepaths = []
    
    # Завантажуємо дані для всіх областей
    for province_id in range(1, 28):
        filepath = download_vhi_data(province_id, output_dir)
        if filepath:
            filepaths.append(filepath)
    
    print(f"Завантажено дані для {len(filepaths)} областей")
    return filepaths

# завантажити дані (якщо нема)
# filepaths = download_all_provinces()

Блок 4: Відповідність ID областей NOAA та українських назв

In [4]:
# Словник перетворення індексів областей
region_mapping = {
    1: 22, 
    2: 24, 
    3: 23,  
    4: 25,  
    5: 3,   
    6: 4,  
    7: 8,   
    8: 19,  
    9: 20,  
    10: 21, 
    11: 9,  
    13: 10, 
    14: 11, 
    15: 12, 
    16: 13, 
    17: 14, 
    18: 15, 
    19: 16, 
    21: 17, 
    22: 18, 
    23: 6,  
    24: 1,  
    25: 2,  
    26: 6,   
    27: 5   
}

region_names = {
    1: "Вінницька",
    2: "Волинська",
    3: "Дніпропетровська",
    4: "Донецька",
    5: "Житомирська",
    6: "Закарпатська",
    7: "Запорізька",
    8: "Івано-Франківська",
    9: "Київська",
    10: "Кіровоградська",
    11: "Луганська",
    12: "Львівська",
    13: "Миколаївська",
    14: "Одеська",
    15: "Полтавська",
    16: "Рівненська",
    17: "Сумська",
    18: "Тернопільська",
    19: "Харківська",
    20: "Херсонська",
    21: "Хмельницька",
    22: "Черкаська",
    23: "Чернівецька",
    24: "Чернігівська",
    25: "Республіка Крим"
}

print("Словники відповідності ID областей та їх назв створено")

Словники відповідності ID областей та їх назв створено


Блок 5: Ф-я для зчитування даних з CSV файлів у DataFrame

In [5]:
def read_vhi_files(data_dir="vhi_data"):

    if not os.path.exists(data_dir):
        print(f"Директорія {data_dir} не існує!")
        return None
    
    csv_files = glob.glob(os.path.join(data_dir, "*.csv"))
    
    if not csv_files:
        print(f"У директорії {data_dir} не знайдено CSV файлів!")
        return None
    
    # Заголовки для DataFrame
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    
    all_data = []
    
    for file in csv_files:
        try:
            # ID області
            province_id = int(file.split('_')[-2])
            
            # Читаю CSV з правильними заголовками
            df = pd.read_csv(file, header=1, names=headers)
            
            # Очищаю дані Year від префіксу
            if isinstance(df.at[0, 'Year'], str) and '<' in df.at[0, 'Year']:
                df.at[0, 'Year'] = df.at[0, 'Year'].split('<')[0].strip()
            
            # Видаляю рядки з VHI = -1
            df = df.drop(df.loc[df['VHI'] == -1].index)
            
            # Видаляю порожній стовпець
            df = df.drop('empty', axis=1)
            
            # Додаємо стовпець з ID області
            df['region_id'] = province_id
            
            # Перетворюю типи даних
            df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
            df['Week'] = pd.to_numeric(df['Week'], errors='coerce')
            
            # Додаю DataFrame до списку
            all_data.append(df)
            
            print(f"Успішно зчитано файл {os.path.basename(file)}")
            
        except Exception as e:
            print(f"Помилка при зчитуванні файлу {file}: {e}")
    
    if not all_data:
        print("Не вдалося зчитати жодного файлу з даними!")
        return None
    
    # Об'єдную всі DataFrame в один
    result_df = pd.concat(all_data, ignore_index=True)
    
    print(f"Створено об'єднаний DataFrame з {len(result_df)} рядками")
    return result_df

Блок 6: Ф-я для зміни індексів областей

In [6]:
def change_region_indices(df):
    if df is None or df.empty:
        print("Отримано порожній DataFrame!")
        return df
    
    # Створюю копію DataFrame
    result_df = df.copy()
    
    # Змінюю індекси областей згідно з таблицею відповідності
    result_df['region_id'] = result_df['region_id'].map(region_mapping)
    
    # Додаю стовпець з назвами областей
    result_df['region_name'] = result_df['region_id'].map(region_names)
    
    print("Індекси областей успішно змінено на українські")
    return result_df

Блок 7: Ф-ї для аналізу даних VHI

In [7]:
def get_vhi_for_region_year(df, region_id, year):

    if df is None or df.empty:
        print("Отримано порожній DataFrame!")
        return None
    
    # Фільтрую дані за областю та роком
    result = df[(df['region_id'] == region_id) & (df['Year'] == year)]
    
    if result.empty:
        print(f"Немає даних для області {region_names.get(region_id, 'Невідома')} за {year} рік")
        return None
    
    print(f"\nДані VHI для області {region_names.get(region_id, 'Невідома')} за {year} рік:")
    print(f"Знайдено {len(result)} тижнів даних")
    
    # Показую перші рядки даних
    table = PrettyTable()
    table.field_names = ["Week", "VHI", "VCI", "TCI"]
    for _, row in result.head(10).iterrows():
        table.add_row([row['Week'], f"{row['VHI']:.2f}", f"{row['VCI']:.2f}", f"{row['TCI']:.2f}"])
    
    print(table)
    
    if len(result) > 10:
        print(f"... і ще {len(result) - 10} рядків")
    
    return result

def get_vhi_extremes(df, region_id, year):

    if df is None or df.empty:
        print("Отримано порожній DataFrame!")
        return None
    
    # Фільтрую дані за областю та роком
    region_data = df[(df['region_id'] == region_id) & (df['Year'] == year)]
    
    if region_data.empty:
        print(f"Немає даних для області {region_names.get(region_id, 'Невідома')} за {year} рік")
        return None
    
    # Рахую екстремуми + сер/мед
    min_vhi = region_data['VHI'].min()
    max_vhi = region_data['VHI'].max()
    mean_vhi = region_data['VHI'].mean()
    median_vhi = region_data['VHI'].median()
    
    # Визначаю тиждень з мін/макс VHI
    min_week = region_data.loc[region_data['VHI'].idxmin(), 'Week']
    max_week = region_data.loc[region_data['VHI'].idxmax(), 'Week']
    
    # Виводжу результати
    print(f"\nЕкстремуми VHI для області {region_names.get(region_id, 'Невідома')} за {year} рік:")
    
    table = PrettyTable()
    table.field_names = ["Показник", "Значення", "Тиждень"]
    table.add_row(["Мінімум", f"{min_vhi:.2f}", min_week])
    table.add_row(["Максимум", f"{max_vhi:.2f}", max_week])
    table.add_row(["Середнє", f"{mean_vhi:.2f}", "-"])
    table.add_row(["Медіана", f"{median_vhi:.2f}", "-"])
    
    print(table)
    
    return {
        'min': min_vhi,
        'max': max_vhi,
        'mean': mean_vhi,
        'median': median_vhi,
        'min_week': min_week,
        'max_week': max_week
    }

def get_vhi_for_years_range(df, region_id, start_year, end_year):
    if df is None or df.empty:
        print("Отримано порожній DataFrame!")
        return None
    
    # Фільтрую дані за областю та діапазоном років
    result = df[(df['region_id'] == region_id) & 
                (df['Year'] >= start_year) & 
                (df['Year'] <= end_year)]
    
    if result.empty:
        print(f"Немає даних для області {region_names.get(region_id, 'Невідома')} за період {start_year}-{end_year}")
        return None
    
    print(f"\nДані VHI для області {region_names.get(region_id, 'Невідома')} за період {start_year}-{end_year}:")
    print(f"Знайдено {len(result)} записів")
    
    # Рахую середні значення VHI за роками
    yearly_stats = result.groupby('Year')['VHI'].agg(['min', 'max', 'mean', 'median']).reset_index()
    
    # Виводжу статистику за роками
    table = PrettyTable()
    table.field_names = ["Рік", "Мінімум VHI", "Максимум VHI", "Середнє VHI", "Медіана VHI"]
    
    for _, row in yearly_stats.iterrows():
        table.add_row([
            int(row['Year']), 
            f"{row['min']:.2f}", 
            f"{row['max']:.2f}", 
            f"{row['mean']:.2f}",
            f"{row['median']:.2f}"
        ])
    
    print(table)
    
    return result

Блок 8: Ф-я для пошуку років з екстремальними посухами

In [8]:
def find_extreme_drought_years(df, threshold_percent=20):

    if df is None or df.empty:
        print("Отримано порожній DataFrame!")
        return None
    
    # Загальна к-ть областей у датасеті
    total_regions = len(set(df['region_id'].dropna()))
    
    # к-ть областей, що відповідає пороговому відсотку
    threshold_count = round(total_regions * threshold_percent / 100)
    
    print(f"\nПошук років з екстремальними посухами (VHI < 15)")
    print(f"Загальна к-ть областей: {total_regions}")
    print(f"Порогова к-ть областей: {threshold_count} (>{threshold_percent}%)")
    
    # Знаходжу всі унікальні роки в даних
    years = sorted(df['Year'].unique())
    
    # Словник для зберігання результатів
    drought_years = {}
    
    # Аналізую кожен рік
    for year in years:
        # Дані для поточного року
        year_data = df[df['Year'] == year]
        
        # Для кожної області визначаю, чи була там екстремальна посуха
        # Область вважається посушливою, якщо будь-яке значення VHI < 15
        drought_regions = []
        
        for region_id in set(year_data['region_id'].dropna()):
            region_min_vhi = year_data[year_data['region_id'] == region_id]['VHI'].min()
            if region_min_vhi < 15:
                drought_regions.append({
                    'region_id': region_id,
                    'region_name': region_names.get(region_id, f"Область {region_id}"),
                    'min_vhi': region_min_vhi
                })
        
        # Якщо к-ть областей перевищує порогове значення
        if len(drought_regions) >= threshold_count:
            drought_years[int(year)] = {
                'count': len(drought_regions),
                'percentage': (len(drought_regions) / total_regions) * 100,
                'regions': drought_regions
            }
    
    if not drought_years:
        print("Не знайдено років з екстремальними посухами для вказаного порогу.")
        return None
    
    # Виводжу результати
    print(f"\nЗнайдено {len(drought_years)} років з екстремальними посухами:")
    
    table = PrettyTable()
    table.field_names = ["Рік", "к-ть областей", "Відсоток", "Області з найнижчим VHI"]
    
    for year, data in sorted(drought_years.items()):
        # Сортую області за значенням VHI (від найнижчого)
        sorted_regions = sorted(data['regions'], key=lambda x: x['min_vhi'])
        
        # Відображаю до 3 областей з найнижчим VHI
        top_regions = [f"{r['region_name']} ({r['min_vhi']:.2f})" for r in sorted_regions[:3]]
        top_regions_str = ", ".join(top_regions)
        
        table.add_row([
            year, 
            data['count'], 
            f"{data['percentage']:.2f}%", 
            top_regions_str + (f" та ще {len(sorted_regions) - 3}..." if len(sorted_regions) > 3 else "")
        ])
    
    print(table)
    
    # Детальний вивід для першого знайденого року
    first_year = list(drought_years.keys())[0]
    first_year_data = drought_years[first_year]
    
    print(f"\nДетальна інформація для {first_year} року:")
    print(f"Всього {first_year_data['count']} областей з посухою ({first_year_data['percentage']:.2f}%):\n")
    
    detail_table = PrettyTable()
    detail_table.field_names = ["Область", "Мінімальний VHI"]
    for region in sorted(first_year_data['regions'], key=lambda x: x['min_vhi']):
        detail_table.add_row([region['region_name'], f"{region['min_vhi']:.2f}"])
    
    print(detail_table)
    
    return drought_years

Блок 9: Основний код для виконання завдання

In [ ]:
# Переконаюсь, що директорія для даних існує
data_dir = "vhi_data"
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
    print(f"Створено директорію {data_dir}")

# Завантаження для всіх областей
# filepaths = download_all_provinces(data_dir)

# Зчитую файли у DataFrame
df = read_vhi_files(data_dir)

# Змінюю індекси областей
if df is not None:
    df = change_region_indices(df)
    
    # Виводжу інформацію про дані
    print("\nЗагальна інформація про дані:")
    print(f"к-ть рядків: {len(df)}")
    print(f"к-ть областей: {df['region_id'].nunique()}")
    
    min_year = df['Year'].min()
    max_year = df['Year'].max()
    print(f"Діапазон років: {min_year}-{max_year}")
    




    # 1. Отримую ряд VHI для області за вказаний рік
    # Можна змінити параметри для отримання даних для інших областей та років
    region_id = 1  # Вінницька область
    year = 2023
    
    print(f"\n--- Аналіз даних для області {region_names.get(region_id, 'Невідома')} за {year} рік ---")
    vhi_data = get_vhi_for_region_year(df, region_id, year)
    
    # 2. Визначаю екстремуми VHI
    extremes = get_vhi_extremes(df, region_id, year)
    
    # 3. Отримую дані VHI для діапазону років
    # Можна змінити параметри для отримання даних для інших областей та діапазонів років
    start_year = 2018
    end_year = 2021
    
    print(f"\n--- Аналіз даних для області {region_names.get(region_id, 'Невідома')} за {start_year}-{end_year} роки ---")
    vhi_years_data = get_vhi_for_years_range(df, region_id, start_year, end_year)
    
    # 4. Знаходжу роки з екстремальними посухами
    # Можна змінити цей параметр для пошуку років з різними порогами
    drought_threshold = 20  # 20% областей
    print(f"\n--- Пошук років з екстремальними посухами (VHI < 15) для більше {drought_threshold}% областей України ---")
    drought_years = find_extreme_drought_years(df, drought_threshold)
else:
    print("Не вдалося створити DataFrame з даними!")

Успішно зчитано файл vhi_id_10_15032025213241.csv
Успішно зчитано файл vhi_id_11_15032025213243.csv
Успішно зчитано файл vhi_id_12_15032025213244.csv
Успішно зчитано файл vhi_id_13_15032025213249.csv
Успішно зчитано файл vhi_id_14_15032025213250.csv
Успішно зчитано файл vhi_id_15_15032025213251.csv
Успішно зчитано файл vhi_id_16_15032025213253.csv
Успішно зчитано файл vhi_id_17_15032025213254.csv
Успішно зчитано файл vhi_id_18_15032025213255.csv
Успішно зчитано файл vhi_id_19_15032025213257.csv
Успішно зчитано файл vhi_id_1_15032025213227.csv
Успішно зчитано файл vhi_id_20_15032025213258.csv
Успішно зчитано файл vhi_id_21_15032025213300.csv
Успішно зчитано файл vhi_id_22_15032025213301.csv
Успішно зчитано файл vhi_id_23_15032025213302.csv
Успішно зчитано файл vhi_id_24_15032025213304.csv
Успішно зчитано файл vhi_id_25_15032025213306.csv
Успішно зчитано файл vhi_id_26_15032025213307.csv
Успішно зчитано файл vhi_id_27_15032025213308.csv
Успішно зчитано файл vhi_id_2_15032025213228.csv
Ус